In [ ]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
os.chdir("/home/yin/DAG")
print(os.getcwd())

/home/yin/DAG


In [2]:
import torch
import torch.nn as nn
from torch.optim import Adam, LBFGS
from nodag.nodag_gumbel_softmax import train_gumbel_sgd
from SCM_data import generate_scm_data
import numpy as np
from numpy.linalg import LinAlgError, inv
from scipy.linalg import sqrtm
import MEC
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [3]:
def nodag_findbest_loss(R_hat, lam=0.5, delta=1e-6, max_steps=5000, tau_start=0.2, tau_end=0.2, times=100):
    best_loss = np.inf
    best_seed = 0
    for t in range(times):
        # if t % (times // 10) == 0:
        #     percent = int(t / times * 100)
        #     print(f"Progress: {percent}%")
        seed = t
        np.random.seed(seed) 
        B_init = np.random.randn(*R_hat.shape)
        B_final,G_final, info = train_gumbel_sgd(
            Rhat_np = R_hat,
            lam = lam,
            delta = delta,
            max_steps = max_steps,
            tau_start = tau_start,
            tau_end = tau_end,
            B_init = B_init
            )
        if info["final_loss"] < best_loss:
            best_loss = info["final_loss"]
            best_likelihood = info["final_likelihood"]
            best_penalty = info["final_penalty"]
            best_seed = seed
            best_G = G_final
            best_B = B_final
    return best_G, best_B, best_loss, best_likelihood, best_penalty, best_seed

In [4]:
def nodag_findbest_likelihood_penalty(
    R_hat, lam=0.5, delta=1e-6, max_steps=5000,
    tau_start=0.2, tau_end=0.2, times=100, eps=1e-3
):
    results = []
    for t in range(times):
        seed = t
        np.random.seed(seed)
        B_init = np.random.randn(*R_hat.shape)
        if t % 100 == 0: print("t = ",t)

        B_final, G_final, info = train_gumbel_sgd(
            Rhat_np=R_hat,
            lam=lam,
            delta=delta,
            max_steps=max_steps,
            tau_start=tau_start,
            tau_end=tau_end,
            B_init=B_init
        )

        results.append({
            "seed": seed,
            "G": G_final,
            "B": B_final,
            "loss": info["final_loss"],
            "likelihood": info["final_likelihood"],
            "penalty": info["final_penalty"],
        })

    min_likelihood = min(r["likelihood"] for r in results)

    candidates = [
        r for r in results
        if abs(r["likelihood"] - min_likelihood) <= eps
    ]

    best = min(candidates, key=lambda r: r["penalty"])

    return (
        best["G"], best["B"],
        best["loss"], best["likelihood"],
        best["penalty"], best["seed"]
    )

In [5]:
# # ER1, find best loss

# from torch.utils.data import Dataset

# from synthetic_dataset import SyntheticDataset
# from synthetic_dataset import dataset_based_on_B

# seed = 1
# # Load dataset
# n, d = 10000, 4
# graph_type, degree = 'ER', 1    # ER1 graph
# B_scale = 1.0
# noise_type = 'gaussian_ev'

# Dataset = SyntheticDataset(n, d, graph_type, degree,
#                         noise_type, B_scale, seed=seed)
# data = Dataset.X
# B_true = Dataset.B
# G_true = Dataset.B_bin

# R_hat = np.cov(data.T)
# d = R_hat.shape[0]
# Omega_true = np.eye(d)
# Theta_hat = inv(R_hat)

# A_true = np.eye(d) - B_true
# # print("A_true = \n",A_true)
# likelihood_true = - 2 * np.log(np.linalg.det(A_true)) + np.trace(A_true.T @ R_hat @ A_true)


# best_G, best_B, best_loss, best_likelihood, best_penalty, best_seed = nodag_findbest_loss(R_hat = R_hat, times = 10)


# print("random seed = ", seed)
# print("likelihood_true = ", likelihood_true)
# print("G_true = \n", G_true)
# print("G_est = \n", best_G)
# # print("Is in MEC: ", MEC.is_in_markov_equiv_class(G_true, best_B))
# print("Final Loss = ", best_loss)
# print("Final penalty = ", best_penalty)
# print("Final likelihood = ", best_likelihood)
# print("seed = ", best_seed)
# print("")

In [6]:
# # ER1, find best penalty in best likelihoods

# from torch.utils.data import Dataset

# from synthetic_dataset import SyntheticDataset
# from synthetic_dataset import dataset_based_on_B

# seed = 1
# # Load dataset
# n, d = 10000, 4
# graph_type, degree = 'ER', 1    # ER1 graph
# B_scale = 1.0
# noise_type = 'gaussian_ev'

# Dataset = SyntheticDataset(n, d, graph_type, degree,
#                         noise_type, B_scale, seed=seed)
# data = Dataset.X
# B_true = Dataset.B
# G_true = Dataset.B_bin

# R_hat = np.cov(data.T)
# d = R_hat.shape[0]
# Omega_true = np.eye(d)
# Theta_hat = inv(R_hat)

# A_true = np.eye(d) - B_true
# # print("A_true = \n",A_true)
# likelihood_true = - 2 * np.log(np.linalg.det(A_true)) + np.trace(A_true.T @ R_hat @ A_true)


# best_G, best_B, best_loss, best_likelihood, best_penalty, best_seed = nodag_findbest_likelihood_penalty(R_hat = R_hat, times = 500)


# print("random seed = ", seed)
# print("likelihood_true = ", likelihood_true)
# print("G_true = \n", G_true)
# print("G_est = \n", best_G)
# # print("Is in MEC: ", MEC.is_in_markov_equiv_class(G_true, best_B))
# print("Final Loss = ", best_loss)
# print("Final penalty = ", best_penalty)
# print("Final likelihood = ", best_likelihood)
# print("seed = ", best_seed)
# print("")

In [7]:
# # Find best loss, seed0-9

# from torch.utils.data import Dataset

# from synthetic_dataset import SyntheticDataset
# from synthetic_dataset import dataset_based_on_B

# for seed in range(10):
#     # Load dataset
#     n, d = 10000, 4
#     graph_type, degree = 'ER', 1    # ER1 graph
#     B_scale = 1.0
#     noise_type = 'gaussian_ev'

#     Dataset = SyntheticDataset(n, d, graph_type, degree,
#                             noise_type, B_scale, seed=seed)
#     data = Dataset.X
#     B_true = Dataset.B
#     G_true = Dataset.B_bin

#     R_hat = np.cov(data.T)
#     d = R_hat.shape[0]
#     Omega_true = np.eye(d)
#     Theta_hat = inv(R_hat)

#     A_true = np.eye(d) - B_true
#     # print("A_true = \n",A_true)
#     likelihood_true = - 2 * np.log(np.linalg.det(A_true)) + np.trace(A_true.T @ R_hat @ A_true)


#     best_G, best_B, best_loss, best_likelihood, best_penalty, best_seed = nodag_findbest_loss(R_hat = R_hat, times = 300)

#     print("random seed = ", seed)
#     print("likelihood_true = ", likelihood_true)
#     print("G_true = \n", G_true)
#     print("G_est = \n", best_G)
#     # print("Is in MEC: ", MEC.is_in_markov_equiv_class(G_true, best_B))
#     print("Final Loss = ", best_loss)
#     print("Final penalty = ", best_penalty)
#     print("Final likelihood = ", best_likelihood)
#     print("seed = ", best_seed)
#     print("")

In [8]:
# Find best penalty in best likelihoods, seed0-9
# d = 4

from torch.utils.data import Dataset

from synthetic_dataset import SyntheticDataset
from synthetic_dataset import dataset_based_on_B

for seed in range(10):
    # Load dataset
    n, d = 10000, 4
    graph_type, degree = 'ER', 1    # ER1 graph
    B_scale = 1.0
    noise_type = 'gaussian_ev'

    Dataset = SyntheticDataset(n, d, graph_type, degree,
                            noise_type, B_scale, seed=seed)
    data = Dataset.X
    B_true = Dataset.B
    G_true = Dataset.B_bin

    R_hat = np.cov(data.T)
    d = R_hat.shape[0]
    Omega_true = np.eye(d)
    Theta_hat = inv(R_hat)

    A_true = np.eye(d) - B_true
    # print("A_true = \n",A_true)
    likelihood_true = - 2 * np.log(np.linalg.det(A_true)) + np.trace(A_true.T @ R_hat @ A_true)


    best_G, best_B, best_loss, best_likelihood, best_penalty, best_seed = nodag_findbest_likelihood_penalty(R_hat = R_hat, times = 300)

    print("random seed = ", seed)
    print("likelihood_true = ", likelihood_true)
    print("G_true = \n", G_true)
    print("G_est = \n", best_G)
    # print("Is in MEC: ", MEC.is_in_markov_equiv_class(G_true, best_B))
    print("Final Loss = ", best_loss)
    print("Final penalty = ", best_penalty)
    print("Final likelihood = ", best_likelihood)
    print("seed = ", best_seed)
    print("")

t =  0


/home/yin/.venv/lib/python3.8/site-packages/torch/autograd/__init__.py:200: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


t =  100


t =  200


random seed =  0
likelihood_true =  3.9726462866418295
G_true = 
 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
G_est = 
 [[-1.  0.  0.  1.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 1.  0.  0. -1.]]
Final Loss =  4.9720383387461675
Final penalty =  1.000000000002756
Final likelihood =  3.9720383387434115
seed =  202

t =  0


...

t =  200


random seed =  8
likelihood_true =  4.0168716736969206
G_true = 
 [[0. 0. 0. 0.]
 [1. 0. 0. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
G_est = 
 [[-1.  1.  0.  1.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 1.  0.  0. -1.]]
Final Loss =  6.016485110961797
Final penalty =  1.9999999867811638
Final likelihood =  4.016485124180633
seed =  183

t =  0


t =  100


t =  200


random seed =  9
likelihood_true =  3.959104150637462
G_true = 
 [[0. 0. 1. 0.]
 [0. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 1. 0.]]
G_est = 
 [[0. 0. 1. 0.]
 [0. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 1. 0.]]
Final Loss =  5.9580652376249015
Final penalty =  2.000000000001206
Final likelihood =  3.9580652376236953
seed =  207



In [ ]:
# Find best penalty in best likelihoods, seed0-9
# d = 6

from torch.utils.data import Dataset

from synthetic_dataset import SyntheticDataset
from synthetic_dataset import dataset_based_on_B

for seed in range(10):
    # Load dataset
    n, d = 100000, 5
    graph_type, degree = 'ER', 1    # ER1 graph
    B_scale = 1.0
    noise_type = 'gaussian_ev'

    Dataset = SyntheticDataset(n, d, graph_type, degree,
                            noise_type, B_scale, seed=seed)
    data = Dataset.X
    B_true = Dataset.B
    G_true = Dataset.B_bin

    R_hat = np.cov(data.T)
    d = R_hat.shape[0]
    Omega_true = np.eye(d)
    Theta_hat = inv(R_hat)

    A_true = np.eye(d) - B_true
    # print("A_true = \n",A_true)
    likelihood_true = - 2 * np.log(np.linalg.det(A_true)) + np.trace(A_true.T @ R_hat @ A_true)


    best_G, best_B, best_loss, best_likelihood, best_penalty, best_seed = nodag_findbest_likelihood_penalty(R_hat = R_hat, times = 300)

    print("random seed = ", seed)
    print("likelihood_true = ", likelihood_true)
    print("G_true = \n", G_true)
    print("G_est = \n", best_G)
    # print("Is in MEC: ", MEC.is_in_markov_equiv_class(G_true, best_B))
    print("Final Loss = ", best_loss)
    print("Final penalty = ", best_penalty)
    print("Final likelihood = ", best_likelihood)
    print("seed = ", best_seed)
    print("")

t =  0


t =  100


t =  200


random seed =  0
likelihood_true =  6.059390325925763
G_true = 
 [[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
G_est = 
 [[ 0.  0.  0.  0.  0.  0.]
 [ 0. -1.  0.  0.  0.  1.]
 [ 0.  1. -1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.]
 [ 0.  0.  1.  0.  0. -1.]]
Final Loss =  9.558528383651488
Final penalty =  3.5000000000069615
Final likelihood =  6.058528383644527
seed =  132

t =  0


t =  100


...

t =  200


random seed =  8
likelihood_true =  6.009763430876404
G_true = 
 [[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]]
G_est = 
 [[-1.  1.  0.  0.  0.  0.]
 [ 1. -1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  1.]
 [ 0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  1.  0. -1.]]
Final Loss =  9.008804224579738
Final penalty =  3.00000000001139
Final likelihood =  6.0088042245683475
seed =  274

t =  0


t =  100


t =  200


random seed =  9
likelihood_true =  5.971202658421388
G_true = 
 [[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0.]]
G_est = 
 [[ 0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.  0.]
 [ 0.  1. -1.  0.  1.  1.]
 [ 0.  0.  1.  0.  0.  0.]
 [ 0.  1.  1.  0.  0.  0.]
 [ 0.  1.  0.  0.  0. -1.]]
Final Loss =  11.970123196355825
Final penalty =  6.000000000006351
Final likelihood =  5.970123196349473
seed =  150



In [10]:
# Find best penalty in best likelihoods, seed0-9
# d = 10

from torch.utils.data import Dataset

from synthetic_dataset import SyntheticDataset
from synthetic_dataset import dataset_based_on_B

for seed in range(10):
    # Load dataset
    n, d = 10000, 10
    graph_type, degree = 'ER', 1    # ER1 graph
    B_scale = 1.0
    noise_type = 'gaussian_ev'

    Dataset = SyntheticDataset(n, d, graph_type, degree,
                            noise_type, B_scale, seed=seed)
    data = Dataset.X
    B_true = Dataset.B
    G_true = Dataset.B_bin

    R_hat = np.cov(data.T)
    d = R_hat.shape[0]
    Omega_true = np.eye(d)
    Theta_hat = inv(R_hat)

    A_true = np.eye(d) - B_true
    # print("A_true = \n",A_true)
    likelihood_true = - 2 * np.log(np.linalg.det(A_true)) + np.trace(A_true.T @ R_hat @ A_true)


    best_G, best_B, best_loss, best_likelihood, best_penalty, best_seed = nodag_findbest_likelihood_penalty(R_hat = R_hat, times = 300)

    print("random seed = ", seed)
    print("likelihood_true = ", likelihood_true)
    print("G_true = \n", G_true)
    print("G_est = \n", best_G)
    # print("Is in MEC: ", MEC.is_in_markov_equiv_class(G_true, best_B))
    print("Final Loss = ", best_loss)
    print("Final penalty = ", best_penalty)
    print("Final likelihood = ", best_likelihood)
    print("seed = ", best_seed)
    print("")

t =  0


t =  100


t =  200


random seed =  0
likelihood_true =  10.0400134065037
G_true = 
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
G_est = 
 [[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.  0.  1.  0.  1.  0.]
 [ 0.  0.  0. -1.  1.  0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0. -1.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  1.  0. -1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  1.  0.  1. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1. -1.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0. -1.]]
Final Loss =  24.03624132891808
Final penalty =  14.000000000037415
Final likelihood =  10.036241328880667
seed =  249

t =  0


t =  100


...

t =  200


random seed =  8
likelihood_true =  10.029179390737745
G_true = 
 [[0. 0. 0. 0. 1. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]
G_est = 
 [[-1.  0.  0.  0.  1.  0.  0.  0.  1.  1.]
 [ 0.  0.  0.  1.  0.  0.  1.  0.  1.  1.]
 [ 0.  0. -1.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -1.  0.  1.  0.  0.]
 [ 0.  0.  1.  0.  0.  0. -1.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  1.  0. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  1.  0.  0.  0. -1.]]
Final Loss =  25.524068206295468
Final penalty =  15.500000000063697
Final likelihood =  10.02406820623177
seed =  90

t =  0


t =  100


t =  200


random seed =  9
likelihood_true =  9.992348082899932
G_true = 
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
G_est = 
 [[-1.  0.  0.  0.  0.  0.  1.  1.  0.  0.]
 [ 0. -1.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  1.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  1. -1.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  1. -1.  1.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. -1.  1.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0. -1.]]
Final Loss =  26.486771258129586
Final penalty =  16.500000001162284
Final likelihood =  9.986771256967304
seed =  170

